In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 model with tracking
model = YOLO('yolov8n.pt')  # Replace with 'yolov8s.pt' or other models if needed

def detect_lane_lines(frame):
    """
    Detect lane lines using edge detection and Hough Transform.
    """
    height, width = frame.shape[:2]
    roi_vertices = np.array([[
        (0, height),              # Bottom-left corner
        (width, height),          # Bottom-right corner
        (width, int(height * 0.55)),  # Top-right (55% height)
        (0, int(height * 0.55))       # Top-left (55% height)
    ]], dtype=np.int32)
    
    # Convert to grayscale and apply Gaussian Blur
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Use Canny edge detection
    edges = cv2.Canny(blur, 50, 150)
    
    # Mask edges to focus on the ROI
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, roi_vertices, 255)
    masked_edges = cv2.bitwise_and(edges, mask)
    
    # Detect lines using Hough Transform
    lines = cv2.HoughLinesP(masked_edges, 1, np.pi / 180, threshold=50, minLineLength=100, maxLineGap=50)
    line_coords = []
    
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            line_coords.append((x1, y1, x2, y2))
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw green lines
    
    return frame, line_coords

def detect_line_violations(track_results, lines):
    """
    Check if tracked vehicles cross detected lane lines.
    """
    violations = []
    for track in track_results:
        if track.boxes:
            for box in track.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
                for line in lines:
                    lx1, ly1, lx2, ly2 = line
                    if line_intersects_box((x1, y1, x2, y2), (lx1, ly1, lx2, ly2)):
                        violations.append((track.id, box))
                        break
    return violations

def line_intersects_box(box, line):
    """
    Check if a line intersects with a bounding box.
    """
    x1, y1, x2, y2 = box
    lx1, ly1, lx2, ly2 = line
    box_edges = [
        ((x1, y1), (x1, y2)),
        ((x1, y2), (x2, y2)),
        ((x2, y2), (x2, y1)),
        ((x2, y1), (x1, y1))
    ]
    for edge in box_edges:
        if lines_intersect(edge[0], edge[1], (lx1, ly1), (lx2, ly2)):
            return True
    return False

def lines_intersect(p1, q1, p2, q2):
    """
    Check if two line segments intersect.
    """
    def orientation(a, b, c):
        val = (b[1] - a[1]) * (c[0] - b[0]) - (b[0] - a[0]) * (c[1] - b[1])
        if val == 0: return 0
        return 1 if val > 0 else 2
    
    def on_segment(a, b, c):
        return min(a[0], b[0]) <= c[0] <= max(a[0], b[0]) and min(a[1], b[1]) <= c[1] <= max(a[1], b[1])
    
    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)
    if o1 != o2 and o3 != o4:
        return True
    if o1 == 0 and on_segment(p1, q1, p2): return True
    if o2 == 0 and on_segment(p1, q1, q2): return True
    if o3 == 0 and on_segment(p2, q2, p1): return True
    if o4 == 0 and on_segment(p2, q2, q1): return True
    return False

def process_video(video_path, output_path=None):
    """
    Process a video for vehicle tracking and lane violation detection.
    """
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out = None
    
    if output_path:
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect lane lines
        frame_with_lines, lines = detect_lane_lines(frame)

        # Perform tracking with YOLOv8
        results = model.track(source=frame, persist=True)
        
        # Detect line violations
        violations = detect_line_violations(results, lines)
        
        # Annotate frame with tracking and violations
        for track in results:
            for box in track.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                color = (0, 255, 0)  # Green for non-violating vehicles
                if (track.id, box) in violations:
                    color = (0, 0, 255)  # Red for violations
                cv2.rectangle(frame_with_lines, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame_with_lines, f'ID {track.id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Show the frame
        cv2.imshow("Lane Violation Detection", frame_with_lines)
        if out:
            out.write(frame_with_lines)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    if out:
        out.release()
    cv2.destroyAllWindows()

# Input and output video paths
video_path = '../1.mp4'
output_path = 'output_video.avi'
process_video(video_path, output_path)


[ WARN:0@31.924] global cap.cpp:779 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.11.0) /io/opencv/modules/videoio/src/cap_images.cpp:415: error: (-215:Assertion failed) !filename_pattern.empty() in function 'CvVideoWriter_Images'


